In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import json
import warnings
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.diagnostic import kstest_normal

import torch
import gpytorch
from gpytorch.kernels import Kernel
from gpytorch.functions import RBFCovariance
from gpytorch.settings import trace_mode
from gpytorch.utils.warnings import GPInputWarning

from dataGen import genData
from utils import trainUtils
from models import ExactGPModel_standard, ExactGPModel_standardScaleK, ExactGPModel_modLengthscale, ExactGPModel_modLengthscaleScaleK

likelihood = gpytorch.likelihoods.GaussianLikelihood()

In [13]:
def gencov(size, std, max, gen):
    if gen==True:
        c=np.random.normal(loc=0, scale=std, size=size)
        for i in range(len(c)):
            if c[i]>max:
                c[i]=max
            elif c[i]<-max:
                c[i]=-max
        return c
    else:
        return None

def gen(size, mean, std):
    return np.random.normal(loc=mean, scale=std, size=size)

def genstd(size, std):
    return np.abs(np.random.normal(loc=0, scale=std, size=size))

def genLinear(c, resVar, idx):
    return c.linear(resVar, idx)

def genPolynomial(c, resVar, idx):
    return c.polynomial(resVar, idx)

def genSin(c, resVar, idx):
    return c.sin(resVar, idx)

def genFeatureSize(min, max):
    return np.random.uniform(min,max,1).astype(int).item()

def genFeatures(SigmaMean, SigmaSigma, SigmaCov, MaxCov, hasCovariances, MeanSampleSize, SigmaSampleSize, minFeatureNb, maxFeatureNb):
    nk=genFeatureSize(minFeatureNb, maxFeatureNb)
    return genData(nk, gen(nk, 0, SigmaMean), genstd(nk, SigmaSigma), gencov(nk-1, SigmaCov, MaxCov, hasCovariances), np.abs(gen(1, MeanSampleSize, SigmaSampleSize)).astype(int).item())

def getFeatureResponseType(data, idx, respVarMean, resVarSigma):
    r=np.random.choice(np.arange(4)) # could change odds
    resVar=np.abs(np.random.normal(loc=respVarMean, scale=resVarSigma, size=1)).item()
    if r==0:
        return data.linear(resVar, idx)
    elif r==1:
        return data.polynomial(resVar, idx)
    elif r==2:
        return data.sin(resVar, idx)
    else:
        return data.addNormalNonPredFeature(idx)

def genDataset(sigmaIntercept, data, respVarMean, resVarSigma, SigmaaddedNoiseAsRate):
    data.intercept(np.random.normal(loc=0, scale=sigmaIntercept, size=1).item())  
    feats=np.arange(data.number_features)
    for i in feats:
        idx=np.random.choice(feats)
        if getFeatureResponseType(data, idx, respVarMean, resVarSigma):
            feats=feats[feats != idx]
            continue
        else:
            if getFeatureResponseType(data, idx, respVarMean, resVarSigma):
                feats=feats[feats != idx]
                continue
            else:
                return False

    addedNoiseAsRate=np.abs(np.random.normal(loc=0, scale=SigmaaddedNoiseAsRate, size=1)).item()
    return trainUtils(data, likelihood, addedNoiseAsRate) 



# run that once
rNoOuputNoReg=[]
rOuputNoReg=[]
rNoOuputReg2=[]
rOuputReg2=[]
rNoOuputReg1=[]
rOuputReg1=[]

with open("rNoOuputNoReg.json", "w") as final:
    json.dump(rNoOuputNoReg, final)
with open("rOuputNoReg.json", "w") as final:
    json.dump(rOuputNoReg, final)
with open("rNoOuputReg2.json", "w") as final:
    json.dump(rNoOuputReg2, final)
with open("rOuputReg2.json", "w") as final:
    json.dump(rOuputReg2, final)
with open("rNoOuputReg1.json", "w") as final:
    json.dump(rNoOuputReg1, final)
with open("rOuputReg1.json", "w") as final:
    json.dump(rOuputReg1, final)

In [14]:
def tojson(l):
    l['sigmay'][0]=float(l['sigmay'][0])
    l['covs']=l['covs'][0].tolist()
    l['lengthscales']=l['lengthscales'][0].tolist()
    return l

In [15]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [ ]:
i=0
while i<50:
    SigmaMean=np.abs(np.random.normal(loc=3, scale=1, size=1)).item()
    SigmaSigma=np.abs(np.random.normal(loc=3, scale=1, size=1)).item()
    SigmaCov=np.abs(np.random.normal(loc=0.12, scale=0.05, size=1)).item()
    MaxCov=0.7
    MeanSampleSize=2500
    SigmaSampleSize=1000
    minFeatureNb=5
    maxFeatureNb=12
    sigmaIntercept=3 
    respVarMean=np.abs(np.random.normal(loc=1, scale=0.5, size=1)).item()
    resVarSigma=np.abs(np.random.normal(loc=0.25, scale=0.1, size=1)).item()
    SigmaaddedNoiseAsRate=np.abs(np.random.normal(loc=0.05, scale=0.1, size=1)).item()
    
    c=genFeatures(SigmaMean, SigmaSigma, SigmaCov, MaxCov, True, MeanSampleSize, SigmaSampleSize, minFeatureNb, maxFeatureNb) #boolean is "hasCovariances"
    train=genDataset(sigmaIntercept, c, respVarMean, resVarSigma, SigmaaddedNoiseAsRate)
    if train==False:
        continue
    else:
        if train.data.checkNormalDistrib(0.01) and len(train.data.nonpred)>=1:
            with open("rNoOuputNoReg.json", "r") as final:
                rNoOuputNoReg=json.load(final)
            with open("rOuputNoReg.json", "r") as final:
                rOuputNoReg=json.load(final)
            with open("rNoOuputReg2.json", "r") as final:
                rNoOuputReg2=json.load(final)
            with open("rOuputReg2.json", "r") as final:
                rOuputReg2=json.load(final)
            with open("rNoOuputReg1.json", "r") as final:
                rNoOuputReg1=json.load(final)
            with open("rOuputReg1.json", "r") as final:
                rOuputReg1=json.load(final)

            train.train(ExactGPModel_standard,True,False,0.2,0,True,40,2,True)
            rNoOuputNoReg.append(tojson(train.results))
            train.clearResults()
            train.train(ExactGPModel_standardScaleK,True,True,0.35,0,True,35,2,True)
            rOuputNoReg.append(tojson(train.results))
            train.clearResults()
            train.train(ExactGPModel_standard,True,False,0.2,0.1,False,40,2,True)
            rNoOuputReg2.append(tojson(train.results))
            train.clearResults()
            train.train(ExactGPModel_standardScaleK,True,True,0.35,0.1,False,35,2,True)
            rOuputReg2.append(tojson(train.results))
            train.clearResults()
            train.train(ExactGPModel_standard,True,False,0.35,0.1,True,40,2,True)
            rNoOuputReg1.append(tojson(train.results))
            train.clearResults()
            train.train(ExactGPModel_standardScaleK,True,True,0.1,1,True,35,2,True)
            rOuputReg1.append(tojson(train.results))
            train.clearResults()

            with open("rNoOuputNoReg.json", "w") as final:
                json.dump(rNoOuputNoReg, final, cls=NpEncoder)
            with open("rOuputNoReg.json", "w") as final:
                json.dump(rOuputNoReg, final, cls=NpEncoder)
            with open("rNoOuputReg2.json", "w") as final:
                json.dump(rNoOuputReg2, final, cls=NpEncoder)
            with open("rOuputReg2.json", "w") as final:
                json.dump(rOuputReg2, final, cls=NpEncoder)
            with open("rNoOuputReg1.json", "w") as final:
                json.dump(rNoOuputReg1, final, cls=NpEncoder)
            with open("rOuputReg1.json", "w") as final:
                json.dump(rOuputReg1, final, cls=NpEncoder)
            i=i+1  
            print(i)  
                
        else:
            continue
